# Spatial analysis of earthquakes

The purpose of this project is to practice the concepts of spatial data science. As you are aware, Tehran is an earthquake-prone city. From the national seismic site, you can get information about earthquakes in different regions. There are two files in the attached files, one related to the earthquakes near Tehran (specifically Damavand) and the other near the earthquakes Qazvin (the areas where these two datasets were prepared overlapped and there is definitely duplicate data in the two files that should be Delete). For each earthquake, the date and time of the earthquake, longitude and latitude (long and lat), depth of the earthquake (in kilometers) and intensity of the earthquake (in Richter) are given.

https://www.dropbox.com/s/lbd3cb8f51wjcfh/Earthquake_Tehran.txt?dl=0

https://www.dropbox.com/s/iw3mtu9vufo24pi/Earthquake_Qazvin.txt?dl=0

Also, in the following address, the location of seismic sites in Iran is specified:

http://irsc.ut.ac.ir/english_txt/Sub%20Network.htm

*: If the maps are not loaded, you can solve the problem by making the notebook trusted.

## Import Libraries

In [750]:
import folium
from folium import Choropleth, Circle, Marker, Icon ,GeoJson
from shapely.ops import cascaded_union
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd

## import Earthquake data

In [751]:
df1 = pd.read_csv('Earthquake_Tehran.txt', delimiter="\t").drop(0)
df2 = pd.read_csv('Earthquake_Qazvin.txt', delimiter="\t").drop(0)
df1['City']='Tehran'
df2['City']='Qazvin'
df=pd.concat([df1,df2])

In [752]:
print(df.shape)
df.head()

(87, 6)


,DATE-TIME,Latitude,Longitude,Depth,Magnitude,City
1,2020-06-02_09:15:51,35.784,52.045,12.1,2.9,Tehran
2,2020-06-01_14:21:16,35.785,52.013,10.6,2.6,Tehran
3,2020-06-01_11:13:54,37.244,53.975,10.0,2.6,Tehran
4,2020-06-01_01:04:36,35.631,52.622,9.2,2.9,Tehran
5,2020-05-31_01:38:48,36.417,51.320,15.4,3.0,Tehran


We see that we do not have null data

In [753]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87 entries, 1 to 41
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DATE-TIME  87 non-null     object
 1   Latitude   87 non-null     object
 2   Longitude  87 non-null     object
 3   Depth      87 non-null     object
 4   Magnitude  87 non-null     object
 5   City       87 non-null     object
dtypes: object(6)
memory usage: 4.8+ KB


change Depth and Magnitude to numeric

In [754]:
df["Depth"] = pd.to_numeric(df["Depth"], downcast="float")
df["Magnitude"] = pd.to_numeric(df["Magnitude"], downcast="float")

## Delete duplicate records

We see that there was a lot of duplicate data

In [755]:
df=df.drop_duplicates(subset=df.columns.difference(['City']))

In [756]:
print(df.shape)
df.head()

(55, 6)


,DATE-TIME,Latitude,Longitude,Depth,Magnitude,City
1,2020-06-02_09:15:51,35.784,52.045,12.1,2.9,Tehran
2,2020-06-01_14:21:16,35.785,52.013,10.6,2.6,Tehran
3,2020-06-01_11:13:54,37.244,53.975,10.0,2.6,Tehran
4,2020-06-01_01:04:36,35.631,52.622,9.2,2.9,Tehran
5,2020-05-31_01:38:48,36.417,51.320,15.4,3.0,Tehran


In [757]:
#creating GeoDataFrame
gdf=gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.Longitude,df.Latitude),crs='epsg:4326')

In [758]:
print(gdf.shape)
gdf.head()

(55, 7)


,DATE-TIME,Latitude,Longitude,Depth,Magnitude,City,geometry
1,2020-06-02_09:15:51,35.784,52.045,12.1,2.9,Tehran,POINT (52.04500 35.78400)
2,2020-06-01_14:21:16,35.785,52.013,10.6,2.6,Tehran,POINT (52.01300 35.78500)
3,2020-06-01_11:13:54,37.244,53.975,10.0,2.6,Tehran,POINT (53.97500 37.24400)
4,2020-06-01_01:04:36,35.631,52.622,9.2,2.9,Tehran,POINT (52.62200 35.63100)
5,2020-05-31_01:38:48,36.417,51.320,15.4,3.0,Tehran,POINT (51.32000 36.41700)


We create a function that colors earthquakes for us according to the city

In [759]:
def color_producer(city):
    if city == 'Tehran':
        return 'red'
    else:
        return 'green'

Earthquake points are displayed on the map of Iran and which city the earthquake is related to is marked in the map as a pop-up. Also, green means the city of Qazvin and red means the city of Tehran

In [760]:
# Create a map
m_1 = folium.Map(location=[32.7,51.3], tiles='Stamen Terrain', zoom_start=5)

# Add points to the map
for idx, row in gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], icon=Icon(color=color_producer(row['City'])),popup=row['City']).add_to(m_1)

# Display the map
m_1


## import seismic sites data

In [761]:
s_df=pd.read_html('StationTable.html',header =0)
s_df = pd.concat(s_df).dropna(axis=0,thresh=2).drop(columns=['No']).fillna('Others')
s_df=s_df.rename({'Station Code': 'Station_Code','Station Name': 'Station_Name','Latitude  (N˚)': 'Latitude','Longitude  (E˚)': 'Longitude','Altitude (m)':'Altitude'}, axis=1)

In [762]:
s_gdf=gpd.GeoDataFrame(s_df,geometry=gpd.points_from_xy(s_df.Longitude,s_df.Latitude),crs='epsg:4326')

In [763]:
print(s_gdf.shape)
s_gdf.head()

(159, 7)


,SubNetwork,Station_Code,Station_Name,Latitude,Longitude,Altitude,geometry
0,Ahwaz,ABH1,Behbahan,30.600,50.253,346.0,POINT (50.25300 30.60000)
1,Ahwaz,AHWZ,Ahwaz,31.330,48.644,19.0,POINT (48.64400 31.33000)
2,Ahwaz,AMIS,Masjed Soleyman,31.665,49.287,442.0,POINT (49.28700 31.66500)
3,Birjand,AFRZ,Afriz,33.425,59.015,1497.0,POINT (59.01500 33.42500)
4,Birjand,DAH,Dahanehchah,32.739,59.868,2341.0,POINT (59.86800 32.73900)


The points of seismic sites are displayed on the geographical map of Iran and the station code is marked in the map as a pop-up.

In [764]:
# Create a map
m_2 = folium.Map(location=[32.7,51.3], tiles='Stamen Terrain', zoom_start=5)

# Add points to the map
for idx, row in s_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], icon=Icon(color='blue',icon='globe', prefix='fa'),popup=row['Station_Name']).add_to(m_2)

# Display the map
m_2


we find the nearest seismic sites station code for each earthquake.

In [765]:
from shapely.ops import nearest_points
pts = s_gdf.geometry.unary_union
def near(point):
     # find the nearest point and return the corresponding Place value
     nearest = s_gdf.geometry == nearest_points(point, pts)[1]
     return s_gdf[nearest].Station_Name.iloc[0]
gdf['Nearest'] = gdf.apply(lambda row: near(row.geometry), axis=1)
gdf.head()

,DATE-TIME,Latitude,Longitude,Depth,Magnitude,City,geometry,Nearest
1,2020-06-02_09:15:51,35.784,52.045,12.1,2.9,Tehran,POINT (52.04500 35.78400),Damavand
2,2020-06-01_14:21:16,35.785,52.013,10.6,2.6,Tehran,POINT (52.01300 35.78500),Damavand
3,2020-06-01_11:13:54,37.244,53.975,10.0,2.6,Tehran,POINT (53.97500 37.24400),Galugah
4,2020-06-01_01:04:36,35.631,52.622,9.2,2.9,Tehran,POINT (52.62200 35.63100),Firuzkuh
5,2020-05-31_01:38:48,36.417,51.320,15.4,3.0,Tehran,POINT (51.32000 36.41700),Pull-Nimvar


In the map, we have visually determined which is the nearest seismic site for each earthquake.
The nearest seismic sites station code has been identified as a pop-up

In [766]:
# Create a map
m_3 = folium.Map(location=[32.7,51.3], tiles='Stamen Terrain', zoom_start=5)

# Add points to the map
for idx, row in gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], icon=Icon(color=color_producer(row['City'])),popup=row['Nearest']).add_to(m_3)

# Display the map
m_3


We add seismic sites in the form of bubble to the map

### Last answer of the first question

In [767]:
# Add a bubble map to the base map
m_4=m_3
for i in range(0,len(s_gdf)):
    Circle(
        location=[s_gdf.iloc[i]['Latitude'], s_gdf.iloc[i]['Longitude']],
        radius=20000,
        color='blue',
        fill=True,
        fill_color='crimson',
        popup=s_gdf.iloc[i]['Station_Name']).add_to(m_4)

# Display the map
m_4


For the damage value of each earthquake, we create a function that returns the radius of the earthquake in proportion to the depth and magnitude.
We know that the bigger the depth of the earthquake have lower the earthquake damage, so we defined different weights for depth and magnitude because they had different scales and minus the depth.

In [768]:
d_weight=1
m_weight=6
def value_producer(Magnitude,Depth):
    res=m_weight*Magnitude-d_weight*Depth
    return res
gdf['Value']=value_producer(gdf['Magnitude'],gdf['Depth'])

This shows the density of earthquake, where red areas have relatively more seismic incidents.

### Last answer of  the second question

In [769]:
# Create a base map
m_5 = folium.Map(location=[32.7,51.3], tiles='Stamen Terrain', zoom_start=5)

# Add a heatmap to the base map
HeatMap(data=gdf[['Latitude', 'Longitude','Value']], radius=20).add_to(m_5)
# Display the map
m_5

We have created three different styles for visual use in maps

In [770]:
style1 = {'fillColor': 'green','color' : 'green', 'lineColor': 'green'}
style2 = {'fillColor': 'red', 'color' : 'red','lineColor': 'red'}
style3 = {'fillColor': 'blue', 'color' : 'blue','lineColor': 'blue'}

We import data of Tehran city and show it in map

You can get polygon coordenates in json for using with googlemaps using openstreetmap. Go to http://nominatim.openstreetmap.org/ search a place like "Tehran"

click on "details"

Look for OSM ID and copy it (control+c)

paste the ID in http://polygons.openstreetmap.fr/index.py and download the polygon

In [771]:
tehran=gpd.read_file('tehran.json')
m_6= folium.Map(location=[35.68,51.43], tiles='Stamen Terrain', zoom_start=8)
folium.GeoJson('tehran.json',style_function=lambda x:style2).add_child(folium.Popup("Tehran boundaries")).add_to(m_6)
# Add points to the map
for idx, row in gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], icon=Icon(color=color_producer(row['City'])),popup=row['Nearest']).add_to(m_6)

# Display the map
m_6

We want to specify a point (or points) on the map that has the following properties.

Located in the urban area of Tehran.

The minimum distance to the epicenter is the maximum

We know that `Voronoi` diagram can be a good option to find exactly these points, but due to project limitations and the use of existing libraries, we implemented the following idea.

We create  buffers to the centers of earthquakes and increase the buffers radius until the intersection of circles unions with the city of Tehran is maximized.

Within a radius of 110 km, the intersection of two areas was maximized.

In [772]:
m_7=m_6
# We change epsg and return it to use the meter scale, also we get warning when use directly
gdf_buffer = gpd.GeoDataFrame(gdf.geometry,geometry=gdf.to_crs(epsg=3763).geometry.buffer(110000).to_crs(epsg=4326))
gdf_buffer_unions = cascaded_union(gdf_buffer.geometry)
GeoJson(gdf_buffer_unions,style_function=lambda x:style1).add_to(m_7)
# Add points to the map
for idx, row in gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], icon=Icon(color=color_producer(row['City'])),popup=row['Nearest']).add_to(m_7)

m_7

We reduce the urban area of Tehran from bubbles to get the farthest point from the sites

In [773]:
d1=gdf.to_crs(epsg=3763).geometry.buffer(110000).to_crs(epsg=4326)
d1 = cascaded_union(d1.geometry)
d2=tehran.to_crs(epsg=3763).geometry.to_crs(epsg=4326)
d2 = cascaded_union(d2.geometry)
d3=d2.difference(d1)

we show the farthest point from the sites wiht blue marker

### Last answer of  the third question

In [774]:
m_8 = folium.Map(location=[35.72,51.15], tiles='Stamen Terrain', zoom_start=9)
GeoJson(d3,style_function=lambda x:style2).add_to(m_8)
# Add points to the map
Marker([d3.centroid.y, d3.centroid.x], icon=Icon(color='blue',icon='safari', prefix='fa'),popup='farthest point').add_to(m_8)
for idx, row in gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], icon=Icon(color=color_producer(row['City'])),popup=row['Nearest']).add_to(m_8)
m_8

In this section, we propose a area for the construction of a new seismic site around Tehran. In other words, we find an area that is at least 50 km away from existing seismic sites and at the same time covers the largest number of earthquakes around Tehran. (Coverage means being less than 25 km from the site).

We create circles with a radius of 25 km to the center of the earthquakes in green and a radius of 50 km to the center of the sites in red.

In [775]:
m_9 = folium.Map(location=[35.8,52], tiles='Stamen Terrain', zoom_start=5)

gdf_buffer = gpd.GeoDataFrame(gdf.geometry,geometry=gdf.to_crs(epsg=3763).geometry.buffer(25000).to_crs(epsg=4326))
gdf_buffer_unions = cascaded_union(gdf_buffer.geometry)
GeoJson(gdf_buffer_unions,style_function=lambda x:style1).add_to(m_9)

s_gdf_buffer = gpd.GeoDataFrame(s_gdf.geometry,geometry=s_gdf.to_crs(epsg=3763).geometry.buffer(50000).to_crs(epsg=4326))
s_gdf_buffer_unions = cascaded_union(s_gdf_buffer.geometry)
GeoJson(s_gdf_buffer_unions,style_function=lambda x:style2).add_to(m_9)
m_9

We subtract the two areas from each other to get the suitable areas

In [776]:
m_10=folium.Map(location=[35.8,52], tiles='Stamen Terrain', zoom_start=7)
diff=gdf_buffer_unions-s_gdf_buffer_unions
GeoJson(diff,style_function=lambda x:style3).add_to(m_10)
HeatMap(data=gdf[['Latitude', 'Longitude','Value']], radius=20).add_to(m_10)
m_10

With the help of the HeattMap , we realized that the Damavand region had many earthquakes ,also closer to Tehran, so we used this area to build a new site.

### Last answer of  the fourth question

In [777]:
m_11 = folium.Map(location=[35.8,52], tiles='Stamen Terrain', zoom_start=8)

GeoJson(diff[3],style_function=lambda x:style2).add_child(folium.Popup("The best place to build a new seismic site around Tehran")).add_to(m_11)
Marker([diff[3].centroid.y, diff[3].centroid.x], icon=Icon(color='blue',icon='safari', prefix='fa'),popup='new seismic site around Tehran').add_to(m_11)

m_11